In [1]:
from langchain_groq import ChatGroq
import os
import config
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = config.groq_key
# os.environ["HTTP_PROXY"] = config.proxy
# os.environ["HTTPS_PROXY"] = config.proxy

In [3]:
llm = ChatGroq(model='llama3-8b-8192', temperature=0)

# ChatMessageHistory

Basic building block memory

In [5]:
from langchain.memory import ChatMessageHistory

In [6]:
history = ChatMessageHistory()

In [8]:
history.add_user_message("Hello nice to meet you")
history.add_ai_message("Nice to meet you too!")

In [10]:
history.messages

[HumanMessage(content='Hello nice to meet you', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Nice to meet you too!', additional_kwargs={}, response_metadata={})]

# Conversation Buffer Memory

In [12]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

In [14]:
memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm, 
                                 memory=memory, 
                                 verbose=True)

C:\Users\ndh00961\AppData\Local\Temp\ipykernel_5980\447071032.py:2: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(llm=llm, memory=memory, verbose=True)


In [15]:
conversation.invoke("Hello nice to meet you")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello nice to meet you
AI:

> Finished chain.


{'input': 'Hello nice to meet you',
 'history': '',
 'response': "Nice to meet you too! I'm an artificial intelligence language model, specifically a variant of the transformer architecture, trained on a massive corpus of text data that includes but is not limited to books, articles, research papers, and online conversations. I'm designed to understand and generate human-like language, and I'm constantly learning and improving my abilities. I'm a bit of a chatterbox, so feel free to ask me anything you'd like to know!"}

In [16]:
conversation.invoke("Tell me an interesting Physics fact")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello nice to meet you
AI: Nice to meet you too! I'm an artificial intelligence language model, specifically a variant of the transformer architecture, trained on a massive corpus of text data that includes but is not limited to books, articles, research papers, and online conversations. I'm designed to understand and generate human-like language, and I'm constantly learning and improving my abilities. I'm a bit of a chatterbox, so feel free to ask me anything you'd like to know!
Human: Tell me an interesting Physics fact
AI:

> Finished chain.


{'input': 'Tell me an interesting Physics fact',
 'history': "Human: Hello nice to meet you\nAI: Nice to meet you too! I'm an artificial intelligence language model, specifically a variant of the transformer architecture, trained on a massive corpus of text data that includes but is not limited to books, articles, research papers, and online conversations. I'm designed to understand and generate human-like language, and I'm constantly learning and improving my abilities. I'm a bit of a chatterbox, so feel free to ask me anything you'd like to know!",
 'response': "Physics is one of my favorite subjects! Did you know that there is a type of particle called a neutrino that is so light it can travel through solid objects, including the Earth, without being slowed down or absorbed? In fact, neutrinos are so light that they can even travel through the entire Earth in just a few seconds! This is because they have a very small mass, which is about 1/100,000th the mass of an electron. Neutrino

In [18]:
print(memory.buffer)

Human: Hello nice to meet you
AI: Nice to meet you too! I'm an artificial intelligence language model, specifically a variant of the transformer architecture, trained on a massive corpus of text data that includes but is not limited to books, articles, research papers, and online conversations. I'm designed to understand and generate human-like language, and I'm constantly learning and improving my abilities. I'm a bit of a chatterbox, so feel free to ask me anything you'd like to know!
Human: Tell me an interesting Physics fact
AI: Physics is one of my favorite subjects! Did you know that there is a type of particle called a neutrino that is so light it can travel through solid objects, including the Earth, without being slowed down or absorbed? In fact, neutrinos are so light that they can even travel through the entire Earth in just a few seconds! This is because they have a very small mass, which is about 1/100,000th the mass of an electron. Neutrinos are created in the core of the

In [19]:
memory.load_memory_variables(())

{'history': "Human: Hello nice to meet you\nAI: Nice to meet you too! I'm an artificial intelligence language model, specifically a variant of the transformer architecture, trained on a massive corpus of text data that includes but is not limited to books, articles, research papers, and online conversations. I'm designed to understand and generate human-like language, and I'm constantly learning and improving my abilities. I'm a bit of a chatterbox, so feel free to ask me anything you'd like to know!\nHuman: Tell me an interesting Physics fact\nAI: Physics is one of my favorite subjects! Did you know that there is a type of particle called a neutrino that is so light it can travel through solid objects, including the Earth, without being slowed down or absorbed? In fact, neutrinos are so light that they can even travel through the entire Earth in just a few seconds! This is because they have a very small mass, which is about 1/100,000th the mass of an electron. Neutrinos are created in

In [21]:
# conversation.memory
import pickle
pick_str = pickle.dumps(conversation.memory)

In [22]:
with open("data/outputs/convo_memory.pkl", "wb") as f:
    f.write(pick_str)

In [23]:
loaded_memory = open("data/outputs/convo_memory.pkl", "rb").read()

relaod_convo = ConversationChain(llm=llm, memory=pickle.loads(loaded_memory))

In [24]:
relaod_convo

ConversationChain(memory=ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Hello nice to meet you', additional_kwargs={}, response_metadata={}), AIMessage(content="Nice to meet you too! I'm an artificial intelligence language model, specifically a variant of the transformer architecture, trained on a massive corpus of text data that includes but is not limited to books, articles, research papers, and online conversations. I'm designed to understand and generate human-like language, and I'm constantly learning and improving my abilities. I'm a bit of a chatterbox, so feel free to ask me anything you'd like to know!", additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me an interesting Physics fact', additional_kwargs={}, response_metadata={}), AIMessage(content="Physics is one of my favorite subjects! Did you know that there is a type of particle called a neutrino that is so light it can travel through solid objects, inc

# Conversation Buffer Window Memory

conversation buffer memory just with the limit to the number of interactions saved,

In [4]:
from langchain_groq import ChatGroq
from langchain.chains import ConversationChain, LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferWindowMemory
llm = ChatGroq(model='llama3-8b-8192', temperature=0)

In [32]:
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(llm=llm, memory=memory)

In [39]:
conversation.invoke("Tell me a match fact with a score")

{'input': 'Tell me a match fact with a score',
 'history': "Human: What was the final score in that match\nAI: I've already told you the answer to that question, and it's still 2-0, with Liverpool FC emerging victorious over Tottenham Hotspur in the 2019 UEFA Champions League Final! I'm happy to provide more information about the match, such as the goals scored by Mohamed Salah and Divock Origi, or the impressive defensive performance by Liverpool's goalkeeper Alisson Becker. Would you like to know more about that match or perhaps explore other sporting events?",
 'response': "The 2018 Formula 1 World Championship finale, the Abu Dhabi Grand Prix! That thrilling race saw Lewis Hamilton and Kimi Räikkönen engage in a neck-and-neck battle, but Hamilton ultimately crossed the finish line first, edging out his Ferrari rival by just 1.3 seconds. The final score was Hamilton in P1, Räikkönen in P2, and Seb Vettel in P3. Would you like to know more about Hamilton's strategy or the battle for 

In [40]:
conversation.invoke("What was the final score in that match")

{'input': 'What was the final score in that match',
 'history': "Human: Tell me a match fact with a score\nAI: The 2018 Formula 1 World Championship finale, the Abu Dhabi Grand Prix! That thrilling race saw Lewis Hamilton and Kimi Räikkönen engage in a neck-and-neck battle, but Hamilton ultimately crossed the finish line first, edging out his Ferrari rival by just 1.3 seconds. The final score was Hamilton in P1, Räikkönen in P2, and Seb Vettel in P3. Would you like to know more about Hamilton's strategy or the battle for 2nd place?",
 'response': 'The final score was Lewis Hamilton in first place, with a marginal 1.3 seconds ahead of Kimi Räikkönen, who settled for second place. The full podium result was: 1. Lewis Hamilton (Mercedes) - 54:36.450, 2. Kimi Räikkönen (Ferrari) - +1.3s, and 3. Sebastian Vettel (Ferrari) - +13.2s.'}

In [38]:
conversation.invoke("What was the final score in that match")

{'input': 'What was the final score in that match',
 'history': "Human: What was the final score in that match\nAI: I've already told you the answer to that question, and it's still 2-0, with Liverpool FC emerging victorious over Tottenham Hotspur in the 2019 UEFA Champions League Final! I'm happy to provide more information about the match, such as the goals scored by Mohamed Salah and Divock Origi, or the impressive defensive performance by Liverpool's goalkeeper Alisson Becker. Would you like to know more about that match or perhaps explore other sporting events?",
 'response': "I've already told you the answer to that question, and it's still 2-0, with Liverpool FC emerging victorious over Tottenham Hotspur in the 2019 UEFA Champions League Final! I'm happy to provide more information about the match, such as the goals scored by Mohamed Salah and Divock Origi, or the impressive defensive performance by Liverpool's goalkeeper Alisson Becker. Would you like to know more about that ma

In [42]:
print(memory.buffer)

Human: What was the final score in that match
AI: The final score was Lewis Hamilton in first place, with a marginal 1.3 seconds ahead of Kimi Räikkönen, who settled for second place. The full podium result was: 1. Lewis Hamilton (Mercedes) - 54:36.450, 2. Kimi Räikkönen (Ferrari) - +1.3s, and 3. Sebastian Vettel (Ferrari) - +13.2s.


In [44]:
memory.load_memory_variables({})

{'history': 'Human: What was the final score in that match\nAI: The final score was Lewis Hamilton in first place, with a marginal 1.3 seconds ahead of Kimi Räikkönen, who settled for second place. The full podium result was: 1. Lewis Hamilton (Mercedes) - 54:36.450, 2. Kimi Räikkönen (Ferrari) - +1.3s, and 3. Sebastian Vettel (Ferrari) - +13.2s.'}

# Convesation Buffer Summary Memory

In [46]:
from langchain.memory import ConversationSummaryBufferMemory

In [47]:
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)

C:\Users\ndh00961\AppData\Local\Temp\ipykernel_2392\3112362828.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)


In [48]:
conversation = ConversationChain(llm=llm,
                                 memory=memory)

In [49]:
conversation.invoke("Tell me a fact about New York")

c:\Users\ndh00961\Desktop\personal_data\personal_code\langchain_personal_exp\lang_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ndh00961\Desktop\personal_data\personal_code\langchain_personal_exp\lang_env\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ndh00961\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to ac

{'input': 'Tell me a fact about New York',
 'history': '',
 'response': 'New York! The Empire State! Did you know that New York is home to the largest public transportation system in the world, with over 472 subway stations and 850 miles of track? It\'s true! The New York City Subway system is one of the most iconic and busiest in the world, with over 6 million passengers per day. In fact, the subway system is so extensive that it\'s often referred to as the "lifeblood" of the city.'}

In [50]:
memory.load_memory_variables({})

{'history': 'System: Current summary: \n\nNew lines of conversation:\nHuman: Tell me a fact about New York\nAI: ()\n\nNew summary:\nAI: New York! The Empire State! Did you know that New York is home to the largest public transportation system in the world, with over 472 subway stations and 850 miles of track? It\'s true! The New York City Subway system is one of the most iconic and busiest in the world, with over 6 million passengers per day. In fact, the subway system is so extensive that it\'s often referred to as the "lifeblood" of the city.'}